# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-13-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-13-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-14 04:26:09,33.93911,67.709953,38716,1420,31638,5658.0,Afghanistan,99.454534,3.667734
1,NaN,NaN,NaN,Albania,2020-09-14 04:26:09,41.15330,20.168300,11353,334,6569,4450.0,Albania,394.502745,2.941954
2,NaN,NaN,NaN,Algeria,2020-09-14 04:26:09,28.03390,1.659600,48254,1612,34037,12605.0,Algeria,110.040712,3.340656
3,NaN,NaN,NaN,Andorra,2020-09-14 04:26:09,42.50630,1.521800,1344,53,943,348.0,Andorra,1739.468064,3.943452
4,NaN,NaN,NaN,Angola,2020-09-14 04:26:09,-11.20270,17.873900,3388,134,1301,1953.0,Angola,10.308441,3.955136


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38304,38324,38398,38494,38520,38544,38572,38606,38641,38716
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9967,10102,10255,10406,10553,10704,10860,11021,11185,11353
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,45773,46071,46364,46653,46938,47216,47488,47752,48007,48254


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1409,1409,1412,1415,1418,1420,1420,1420,1420,1420
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,306,312,316,319,321,322,324,327,330,334
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1539,1549,1556,1562,1571,1581,1591,1599,1605,1612


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,29713,30082,30537,30557,30715,31048,31129,31154,31234,31638
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5882,5976,6106,6186,6239,6284,6346,6443,6494,6569
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,32259,32481,32745,32985,33183,33379,33562,33723,33875,34037


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1355,1371,1377,1383,1385,1398,1413,1420,1432,1442
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4513,4542,4569,4586,4609,4639,4678,4722,4752,4781
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,616,616,616,617,617,618,618,618,620,622


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,23,23,23,23,23,23,23,23,23,23
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,42,42,42,42,42,42,42,43,43,43
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-09-14 04:26:09,32.539527,-86.644082,1442,23,0,1419.0,"Autauga, Alabama, US",2581.037785,1.595007
668,1075.0,Lamar,Alabama,US,2020-09-14 04:26:09,33.779950,-88.096680,292,2,0,290.0,"Lamar, Alabama, US",2115.175661,0.684932
669,1077.0,Lauderdale,Alabama,US,2020-09-14 04:26:09,34.901719,-87.656247,1527,30,0,1497.0,"Lauderdale, Alabama, US",1646.734031,1.964637


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6519573,194071,2451406
India,4754356,78586,3702595
Brazil,4330455,131625,3723206


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6519573,194071,2451406,3874096,2020-09-14 04:26:09,37.923092,-91.345036
India,4754356,78586,3702595,973175,2020-09-14 04:26:09,23.088275,81.806127
Brazil,4330455,131625,3723206,475624,2020-09-14 04:26:09,-12.669522,-48.480493
Russia,1059024,18517,873684,166823,2020-09-14 04:26:09,54.546312,62.120860
Peru,722832,30526,559321,132985,2020-09-14 04:26:09,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,761728,14378,0
Texas,681885,14405,0
Florida,663994,12608,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,761728,14378,0,747350,2020-09-14 04:26:09,37.843962,-120.728594
Texas,681885,14405,0,667480,2020-09-14 04:26:09,31.660643,-98.653069
Florida,663994,12608,0,651386,2020-09-14 04:26:09,28.940755,-82.700744
New York,444365,33023,0,411342,2020-09-14 04:26:09,42.544151,-75.474183
Georgia,294314,6333,0,287981,2020-09-14 04:26:09,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,253985,6208,0
Florida,Miami-Dade,164086,2880,0
Arizona,Maricopa,137507,3154,0
Illinois,Cook,134801,5133,0
Texas,Harris,116172,2420,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,253985,6208,0,247777,2020-09-14 04:26:09,34.308284,-118.228241,6037.0
Florida,Miami-Dade,164086,2880,0,161206,2020-09-14 04:26:09,25.611236,-80.551706,12086.0
Arizona,Maricopa,137507,3154,0,134353,2020-09-14 04:26:09,33.348359,-112.491815,4013.0
Illinois,Cook,134801,5133,0,129668,2020-09-14 04:26:09,41.841448,-87.816588,17031.0
Texas,Harris,116172,2420,0,113752,2020-09-14 04:26:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,38606,11021,47752,1344,3279,95,535705,45503,26607,31827,...,364088,1773,46160,58663,1060,29256,10,2007,13323,7479
2020-09-12,38641,11185,48007,1344,3335,95,546481,45675,26651,32696,...,367592,1780,46721,59630,1060,29906,10,2009,13466,7508
2020-09-13,38716,11353,48254,1344,3388,95,555537,45862,26692,33159,...,370930,1808,47287,60540,1063,30574,10,2011,13539,7526


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,1420,327,1599,53,131,3,11148,909,803,750,...,41703,45,377,468,35,204,1,582,306,224
2020-09-12,1420,330,1605,53,132,3,11263,911,810,754,...,41712,45,384,477,35,210,1,582,312,224
2020-09-13,1420,334,1612,53,134,3,11352,916,816,756,...,41717,45,390,485,35,221,1,583,312,224


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,31154,6443,33723,943,1288,91,400121,41434,23327,26257,...,1848,1490,42857,46960,902,19788,8,1211,11899,5660
2020-09-12,31234,6494,33875,943,1289,91,409771,41605,23338,26579,...,1880,1502,43359,47729,910,19979,8,1211,12007,5675
2020-09-13,31638,6569,34037,943,1301,91,419513,41659,23463,26760,...,2146,1513,43866,48644,918,20082,8,1212,12260,5678


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1355,1371,1377,1383,1385,1398,1413,1420,1432,1442
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,278,280,282,282,282,282,283,288,289,292
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1454,1467,1470,1479,1485,1490,1504,1512,1518,1527
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,488,501,502,505,506,510,515,526,529,532
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,3417,3466,3488,3511,3529,3553,3560,3585,3593,3613


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,136703,6104,0,207523,67911,754905,60462,54326,18559,49,...,169859,674943,56675,1668,1201,131571,79011,12177,86250,4264
2020-09-12,137646,6207,0,208128,69710,758933,60876,54326,18726,49,...,170891,677625,57247,1677,1211,132868,79476,12528,87603,4297
2020-09-13,138755,6268,0,208512,70219,761728,61293,54326,18849,49,...,171824,681885,57875,1684,1220,133742,79826,12705,89185,4346


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-11        1420    4722     618  574   1109     551    863    2679   
2020-09-12        1432    4752     620  576   1114     552    865    2728   
2020-09-13        1442    4781     622  578   1121     558    866    2773   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-11          862      380  ...     184      9      197       53   
2020-09-12          863      392  ...     187      9      199       53   
2020-09-13          863      407  ...     187      9      203       54   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-11            320   462   310          0      111     21  
2020-09-12            320   465   310          0      111     23  
2020-09-13            318   470   311          2      111     23  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,2333,43,0,5288,953,14230,1985,4480,613,0,...,2025,14227,431,58,18,2711,1991,264,1197,42
2020-09-12,2350,44,0,5315,969,14327,1988,4480,613,0,...,2064,14332,433,58,19,2722,1991,266,1209,42
2020-09-13,2351,44,0,5322,981,14378,1988,4480,615,0,...,2078,14405,433,58,19,2724,1991,267,1210,42


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-11          23      43       7    6     12      13     37      31   
2020-09-12          23      43       7    6     13      13     37      31   
2020-09-13          23      43       7    6     13      13     37      31   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-11           40       11  ...       1      1        1        1   
2020-09-12           40       11  ...       1      1        1        1   
2020-09-13           40       11  ...       1      1        1        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-11              2     1     2          0        6      0  
2020-09-12              2     1     2          0        6      0  
2020-09-13              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.000881,0.014825,0.005559,0.033051,0.019273,0.0,0.021952,0.003905,0.001619,0.018562,...,0.009830,0.007959,0.015108,0.014527,0.000944,0.020653,0.0,0.001997,0.008249,0.003489
2020-09-12,0.000907,0.014881,0.005340,0.000000,0.017078,0.0,0.020116,0.003780,0.001654,0.027304,...,0.009624,0.003948,0.012153,0.016484,0.000000,0.022218,0.0,0.000997,0.010733,0.003878
2020-09-13,0.001941,0.015020,0.005145,0.000000,0.015892,0.0,0.016571,0.004094,0.001538,0.014161,...,0.009081,0.015730,0.012114,0.015261,0.002830,0.022337,0.0,0.000996,0.005421,0.002397


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.0,0.009259,0.005028,0.0,0.007692,0.0,0.022096,0.003311,0.007528,0.002674,...,0.000144,0.0,0.018919,0.017391,0.0,0.030303,0.0,0.003448,0.020000,0.009009
2020-09-12,0.0,0.009174,0.003752,0.0,0.007634,0.0,0.010316,0.002200,0.008717,0.005333,...,0.000216,0.0,0.018568,0.019231,0.0,0.029412,0.0,0.000000,0.019608,0.000000
2020-09-13,0.0,0.012121,0.004361,0.0,0.015152,0.0,0.007902,0.005488,0.007407,0.002653,...,0.000120,0.0,0.015625,0.016771,0.0,0.052381,0.0,0.001718,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.000803,0.015285,0.004797,0.00533,0.008614,0.0,0.025694,0.004875,0.012588,0.008217,...,0.001626,0.004043,0.008187,0.018103,0.010078,0.051379,0.0,0.000000,0.000000,0.004437
2020-09-12,0.002568,0.007916,0.004507,0.00000,0.000776,0.0,0.024118,0.004127,0.000472,0.012263,...,0.017316,0.008054,0.011713,0.016376,0.008869,0.009652,0.0,0.000000,0.009076,0.002650
2020-09-13,0.012935,0.011549,0.004782,0.00000,0.009310,0.0,0.023774,0.001298,0.005356,0.006810,...,0.141489,0.007324,0.011693,0.019171,0.008791,0.005155,0.0,0.000826,0.021071,0.000529


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,0.008394,0.015303,NaN,0.002517,0.016571,0.005252,0.005103,0.004307,0.005036,0.0,...,0.009641,0.008671,0.011710,0.004214,0.000000,0.008516,0.006933,0.013399,0.016128,0.015480
2020-09-12,0.006898,0.016874,NaN,0.002915,0.026491,0.005336,0.006847,0.000000,0.008998,0.0,...,0.006076,0.003974,0.010093,0.005396,0.008326,0.009858,0.005885,0.028825,0.015687,0.007739
2020-09-13,0.008057,0.009828,NaN,0.001845,0.007302,0.003683,0.006850,0.000000,0.006568,0.0,...,0.005460,0.006287,0.010970,0.004174,0.007432,0.006578,0.004404,0.014128,0.018059,0.011403


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-11      0.004954  0.009406  0.000000  0.014134  0.009099  0.001818   
2020-09-12      0.008451  0.006353  0.003236  0.003484  0.004509  0.001815   
2020-09-13      0.006983  0.006103  0.003226  0.003472  0.006284  0.010870   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-11      0.000000  0.010181  0.004662  0.010638  ...  0.010989   
2020-09-12      0.002317  0.018290  0.001160  0.031579  ...  0.016304   
2020-09-13      0.001156  0.016496  0.000000  0.038265  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-11      0.285714  0.031414  0.039216   0.003135  0.019868  0.000000   
2020-09-12      0.000000  0.010152  0.000000   0.000000  0.006494  0.000000   
2020-09-13      0.000000  0.020101  0.018868  -0.006250  0.010753  0.003226   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2020-09-11            NaN      0.0  0.000000  
2020-09-12            NaN      0.0  0.095238  
2020-09-13            inf      0.0  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,0.013907,0.023810,NaN,0.002845,0.013830,0.010869,0.003032,0.000447,0.000000,NaN,...,0.018612,0.021321,0.002326,0.0,0.000000,0.001108,0.003023,0.023256,0.003353,0.0
2020-09-12,0.007287,0.023256,NaN,0.005106,0.016789,0.006817,0.001511,0.000000,0.000000,NaN,...,0.019259,0.007380,0.004640,0.0,0.055556,0.004058,0.000000,0.007576,0.010025,0.0
2020-09-13,0.000426,0.000000,NaN,0.001317,0.012384,0.003560,0.000000,0.000000,0.003263,NaN,...,0.006783,0.005093,0.000000,0.0,0.000000,0.000735,0.000000,0.003759,0.000827,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga  Baldwin Barbour Bibb    Blount Bullock Butler Calhoun   
2020-09-11         0.0  0.02381     0.0  0.0  0.000000     0.0    0.0     0.0   
2020-09-12         0.0  0.00000     0.0  0.0  0.083333     0.0    0.0     0.0   
2020-09-13         0.0  0.00000     0.0  0.0  0.000000     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-11          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-12          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-13          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-11            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-12            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-13            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.000860,0.014626,0.005741,0.021754,0.023913,0.000021,0.022585,0.003775,0.001817,0.018497,...,0.008829,0.009883,0.013394,0.017364,0.001377,0.022356,2.297724e-26,0.002052,0.008522,0.006249
2020-09-12,0.000884,0.014753,0.005541,0.010877,0.020496,0.000010,0.021350,0.003777,0.001735,0.022901,...,0.009227,0.006916,0.012774,0.016924,0.000689,0.022287,1.148862e-26,0.001525,0.009628,0.005063
2020-09-13,0.001412,0.014887,0.005343,0.005439,0.018194,0.000005,0.018961,0.003936,0.001637,0.018531,...,0.009154,0.011323,0.012444,0.016092,0.001759,0.022312,5.744310e-27,0.001260,0.007524,0.003730


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.000412,0.007750,0.005535,8.955025e-12,0.016941,7.061231e-46,0.022981,0.002622,0.009467,0.002205,...,0.000235,0.000090,0.016932,0.017701,0.000078,0.027689,0.0,0.003855,0.011860,0.009813
2020-09-12,0.000206,0.008462,0.004644,4.477512e-12,0.012287,3.530615e-46,0.016648,0.002411,0.009092,0.003769,...,0.000225,0.000045,0.017750,0.018466,0.000039,0.028550,0.0,0.001928,0.015734,0.004907
2020-09-13,0.000103,0.010292,0.004503,2.238756e-12,0.013719,1.765308e-46,0.012275,0.003950,0.008250,0.003211,...,0.000173,0.000023,0.016687,0.017619,0.000019,0.040466,0.0,0.001823,0.007867,0.002453


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-11,0.003139,0.012631,0.005355,0.003487,0.014676,2.745128e-06,0.026397,0.006089,0.009734,0.008949,...,0.003539,0.003903,0.008347,0.018550,0.012290,0.038424,1.346323e-28,0.001309,0.002320,0.008870
2020-09-12,0.002854,0.010273,0.004931,0.001744,0.007726,1.372564e-06,0.025257,0.005108,0.005103,0.010606,...,0.010427,0.005979,0.010030,0.017463,0.010580,0.024038,6.731613e-29,0.000655,0.005698,0.005760
2020-09-13,0.007894,0.010911,0.004857,0.000872,0.008518,6.862821e-07,0.024516,0.003203,0.005230,0.008708,...,0.075958,0.006651,0.010862,0.018317,0.009686,0.014597,3.365807e-29,0.000740,0.013384,0.003144


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,0.007754,0.015196,NaN,0.002252,0.011302,0.004900,0.004653,0.003899,0.005468,4.442760e-55,...,0.008663,0.006439,0.008946,0.003236,0.002080,0.008337,0.005987,0.013990,0.015147,0.013376
2020-09-12,0.007326,0.016035,NaN,0.002584,0.018896,0.005118,0.005750,0.001950,0.007233,2.221380e-55,...,0.007369,0.005206,0.009519,0.004316,0.005203,0.009098,0.005936,0.021407,0.015417,0.010558
2020-09-13,0.007692,0.012931,NaN,0.002214,0.013099,0.004400,0.006300,0.000975,0.006901,1.110690e-55,...,0.006414,0.005746,0.010245,0.004245,0.006317,0.007838,0.005170,0.017768,0.016738,0.010981


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-11      0.006744  0.008234  0.000240  0.009323  0.010158  0.002589   
2020-09-12      0.007597  0.007293  0.001738  0.006404  0.007333  0.002202   
2020-09-13      0.007290  0.006698  0.002482  0.004938  0.006808  0.006536   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-11      0.000960  0.010233  0.002577  0.010412  ...  0.014450   
2020-09-12      0.001638  0.014262  0.001869  0.020996  ...  0.015377   
2020-09-13      0.001397  0.015379  0.000934  0.029630  ...  0.007689   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-11      0.142857  0.032103  0.023670   0.005898  0.012051  0.001489   
2020-09-12      0.071429  0.021128  0.011835   0.002949  0.009273  0.000744   
2020-09-13      0.035714  0.020614  0.015351  -0.001650  0.010013  0.001985   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-11      -0.495957  0.000187  0.007063  
2020-09-12      -0.495957  0.000093  0.051151  
2020-09-13      -0.495957  0.000047  0.025575  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-11,0.009297,0.012373,NaN,0.003360,0.013201,0.009072,0.002117,0.000532,0.001696,NaN,...,0.020182,0.012772,0.004190,1.286634e-13,0.004820,0.002294,0.004030,0.017368,0.005542,0.000201
2020-09-12,0.008292,0.017814,NaN,0.004233,0.014995,0.007945,0.001814,0.000266,0.000848,NaN,...,0.019721,0.010076,0.004415,6.433172e-14,0.030188,0.003176,0.002015,0.012472,0.007784,0.000100
2020-09-13,0.004359,0.008907,NaN,0.002775,0.013690,0.005752,0.000907,0.000133,0.002055,NaN,...,0.013252,0.007585,0.002208,3.216586e-14,0.015094,0.001955,0.001008,0.008116,0.004305,0.000050


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-11      0.000022  0.012220  9.980688e-09 -0.000227  0.000403   
2020-09-12      0.000011  0.006110  4.990344e-09 -0.000114  0.041868   
2020-09-13      0.000006  0.003055  2.495172e-09 -0.000057  0.020934   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun  Chambers      Cherokee  ...   
2020-09-11      2.709302e-09  0.000109 -0.000013  0.000101  3.405760e-06  ...   
2020-09-12      1.354651e-09  0.000054 -0.000007  0.000050  1.702880e-06  ...   
2020-09-13      6.773255e-10  0.000027 -0.000003  0.000025  8.514400e-07  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-09-11         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-12         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-13         0.0    0.0      0.0      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-11      -0.499985      0.0    NaN  
2020-09-12      -0.499985      0.0    NaN  
2020-09-13      -0.499985      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200913,AK,7193,394020,NaN,35.0,NaN,NaN,NaN,8.0,...,401213,0,0,f467ac0e20900a51da5e02bc4fb53260331b017c,0,0,0,0,0,NaN
1,20200913,AL,138755,884432,NaN,790.0,15527.0,NaN,1610.0,NaN,...,1023187,1,0,b84d1499d4e7dde4dca550ed4ab983ad7a428a9e,0,0,0,0,0,NaN
2,20200913,AR,69710,758511,NaN,381.0,4719.0,160.0,NaN,83.0,...,828221,12,41,a6dba58bc6a974f8606aa27cbb710e0566b99195,0,0,0,0,0,NaN
3,20200913,AS,0,1571,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,3ae357e9638017e50197c3bf2bd193a2569941a7,0,0,0,0,0,NaN
4,20200913,AZ,208512,1087625,NaN,525.0,21773.0,171.0,NaN,87.0,...,1296137,7,0,e5edb0962ba406565640dc14c0105876dbbccc1f,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200913,AK,7193.0,394020.0,NaN,35.0,NaN,NaN,NaN,8.0,...,401213,0,0,f467ac0e20900a51da5e02bc4fb53260331b017c,0,0,0,0,0,NaN
1,20200913,AL,138755.0,884432.0,NaN,790.0,15527.0,NaN,1610.0,NaN,...,1023187,1,0,b84d1499d4e7dde4dca550ed4ab983ad7a428a9e,0,0,0,0,0,NaN
2,20200913,AR,69710.0,758511.0,NaN,381.0,4719.0,160.0,NaN,83.0,...,828221,12,41,a6dba58bc6a974f8606aa27cbb710e0566b99195,0,0,0,0,0,NaN
3,20200913,AS,0.0,1571.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,3ae357e9638017e50197c3bf2bd193a2569941a7,0,0,0,0,0,NaN
4,20200913,AZ,208512.0,1087625.0,NaN,525.0,21773.0,171.0,NaN,87.0,...,1296137,7,0,e5edb0962ba406565640dc14c0105876dbbccc1f,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-13,AK,7193,394020,NaN,35.0,NaN,NaN,NaN,8.0,NaN,...,401213,0,0,f467ac0e20900a51da5e02bc4fb53260331b017c,0,0,0,0,0,NaN
2020-09-13,AL,138755,884432,NaN,790.0,15527.0,NaN,1610.0,NaN,892.0,...,1023187,1,0,b84d1499d4e7dde4dca550ed4ab983ad7a428a9e,0,0,0,0,0,NaN
2020-09-13,AR,69710,758511,NaN,381.0,4719.0,160.0,NaN,83.0,601.0,...,828221,12,41,a6dba58bc6a974f8606aa27cbb710e0566b99195,0,0,0,0,0,NaN
2020-09-13,AS,0,1571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,3ae357e9638017e50197c3bf2bd193a2569941a7,0,0,0,0,0,NaN
2020-09-13,AZ,208512,1087625,NaN,525.0,21773.0,171.0,NaN,87.0,NaN,...,1296137,7,0,e5edb0962ba406565640dc14c0105876dbbccc1f,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-13,AK,7193.0,394020.0,NaN,35.0,NaN,NaN,NaN,8.0,NaN,...,401213,0,0,f467ac0e20900a51da5e02bc4fb53260331b017c,0,0,0,0,0,NaN
2020-09-13,AL,138755.0,884432.0,NaN,790.0,15527.0,NaN,1610.0,NaN,892.0,...,1023187,1,0,b84d1499d4e7dde4dca550ed4ab983ad7a428a9e,0,0,0,0,0,NaN
2020-09-13,AR,69710.0,758511.0,NaN,381.0,4719.0,160.0,NaN,83.0,601.0,...,828221,12,41,a6dba58bc6a974f8606aa27cbb710e0566b99195,0,0,0,0,0,NaN
2020-09-13,AS,0.0,1571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,3ae357e9638017e50197c3bf2bd193a2569941a7,0,0,0,0,0,NaN
2020-09-13,AZ,208512.0,1087625.0,NaN,525.0,21773.0,171.0,NaN,87.0,NaN,...,1296137,7,0,e5edb0962ba406565640dc14c0105876dbbccc1f,0,0,0,0,0,NaN
